In [1]:
import os
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from diffusers import UNet2DModel, DDPMScheduler
import imageio
import torchvision.utils as vutils
import torch.nn.functional as F

/data/home/saisathvik/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATASET_PATH = "img_align_celeba"   

SAVE_DIR = "checkpoints"
SAMPLE_DIR = "samples"

os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(SAMPLE_DIR, exist_ok=True)

In [3]:
class CelebADataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_files = sorted(os.listdir(root_dir))
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image

In [4]:
image_size = 64
batch_size = 256

transform = transforms.Compose([
    transforms.CenterCrop(178),
    transforms.Resize((64,64)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3,[0.5]*3)
])


dataset = CelebADataset(DATASET_PATH, transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

print("Total images:", len(dataset))

Total images: 202599


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

model = UNet2DModel(
    sample_size=64,
    in_channels=3,
    out_channels=3,
    layers_per_block=2,
    block_out_channels=(64, 128, 128, 256),
    down_block_types=("DownBlock2D","DownBlock2D","AttnDownBlock2D","DownBlock2D"),
    up_block_types=("UpBlock2D","AttnUpBlock2D","UpBlock2D","UpBlock2D"),
).to(device)

noise_scheduler = DDPMScheduler(num_train_timesteps=1000)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

cuda


In [6]:
torch.manual_seed(42)
fixed_noise = torch.randn((9, 3, 64, 64)).to(device)

In [7]:
@torch.no_grad()
def sample_images(epoch):

    model.eval()

    sample = fixed_noise.clone()

    # reverse diffusion
    for t in reversed(range(1000)):

        t_tensor = torch.full((9,), t, device=device, dtype=torch.long)

        noise_pred = model(sample, t_tensor).sample
        sample = noise_scheduler.step(noise_pred, t, sample).prev_sample

    # [-1,1] -> [0,1]
    imgs = (sample.clamp(-1, 1) + 1) / 2
    imgs = imgs.cpu()

    # 3x3 GRID
    grid = vutils.make_grid(imgs, nrow=3, padding=2)
    grid = grid.permute(1, 2, 0).numpy()

    plt.figure(figsize=(6,6))
    plt.axis("off")
    plt.imshow(grid)
    plt.savefig(f"{SAMPLE_DIR}/epoch_{epoch:03d}.png", bbox_inches='tight', pad_inches=0)
    plt.close()

    print(f"Saved samples for epoch {epoch}")

In [ ]:
epochs = 60

for epoch in range(epochs):

    model.train()
    pbar = tqdm(dataloader)

    for batch in pbar:

        # Load clean images
        clean_images = batch.to(device, non_blocking=True)

        # Sample noise
        noise = torch.randn_like(clean_images)

        # Random timestep for each image
        timesteps = torch.randint(
            0, 1000, (clean_images.shape[0],), device=device
        ).long()

    
        # Forward diffusion (add noise)
        noisy_images = noise_scheduler.add_noise(
            clean_images, noise, timesteps
        )

        noise_pred = model(noisy_images, timesteps).sample

        loss = F.mse_loss(noise_pred, noise)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pbar.set_description(f"Epoch {epoch} | Loss: {loss.item():.4f}")

    model.save_pretrained(f"{SAVE_DIR}/epoch_{epoch}")

    sample_images(epoch)

print("Training Finished!")

Epoch 0 | Loss: 0.0310: 100%|██████████| 792/792 [05:57<00:00,  2.21it/s]


Saved samples for epoch 0


Epoch 1 | Loss: 0.0168: 100%|██████████| 792/792 [05:56<00:00,  2.22it/s]


Saved samples for epoch 1


Epoch 2 | Loss: 0.0162: 100%|██████████| 792/792 [05:56<00:00,  2.22it/s]


...

Saved samples for epoch 57


Epoch 58 | Loss: 0.0149: 100%|██████████| 792/792 [05:56<00:00,  2.22it/s]


Saved samples for epoch 58


Epoch 59 | Loss: 0.0177: 100%|██████████| 792/792 [05:57<00:00,  2.22it/s]


Saved samples for epoch 59
Training Finished!
